# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict, Counter
#natural language processing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer

stop_words = set(stopwords.words('english'))

#machine learning
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_selection import chi2, SelectKBest
import re
import string

import matplotlib.pyplot as plt
import seaborn as sns
import math

import pickle
import warnings
from sqlalchemy import create_engine

warnings.filterwarnings('ignore')
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster', con=engine)

In [3]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def find_url(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) 
    return url 

def tokenize(text):
    """
    Convertes text to words
    """
    urls = find_url(text)
    for url in  urls:
        text = text.replace(url, "URL")
        
    #case folding - normalizing
    text = text.lower()
    # remove punctuations
    words = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text))
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # to reduce our words to a single word, better than stemming
    words = [WordNetLemmatizer().lemmatize(word) for word in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
pipeline.fit(X_train, y_train)

In [14]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#list of categories
categories = y.columns.tolist()

In [12]:

def clf_report_mult(y_test, y_pred):
    clf_results = pd.DataFrame(columns=['category', 'precision', 'recall', 'f1_score'])
    for i in range(0, len(categories)):
        precision, recall, f1_score, support = precision_recall_fscore_support(y_test[categories[i]], y_pred[:,i], average='weighted')
        clf_results = clf_results.append({'category': categories[i], 'precision': precision, 'recall': recall, 'f1_score': f1_score}, ignore_index=True)
    return clf_results

clf_report_mult(y_test, y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = { 
                'clf__estimator__n_estimators': [25, 50],
                'clf__estimator__max_depth': [10, 25]
                }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.10813903590330425, total= 1.8min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.13617508224860536, total= 1.8min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.11917024320457797, total= 1.8min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.1407523959376341, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.16034902016878844, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.15236051502145923, total= 1.9min
[CV] clf__estimator__max_depth=25, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=25, clf__estimator__n_estimators=25, score=0.16950364754684594, total= 1.9min
[CV] clf__estimator__max_depth=25, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=25, clf__estimator__n_estimators=25, score=0.1879559433557431, t

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 38.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [25, 50], 'clf__estimator__max_depth': [10, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred = cv.predict(X_test)

clf_report_mult(y_test, y_pred)

,category,precision,recall,f1_score
0,related,0.794359,0.687834,0.709763
1,request,0.884664,0.872044,0.876903
2,offer,0.992007,0.995995,0.993997
3,aid_related,0.766799,0.767925,0.767175
4,medical_help,0.915782,0.911709,0.913648
5,medical_products,0.939627,0.938978,0.939300
6,search_and_rescue,0.960947,0.967963,0.963878
7,security,0.972910,0.981693,0.976565
8,military,0.967706,0.965866,0.966737
9,child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('svd', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ])


In [10]:
parameters = { 
                'clf__estimator__n_estimators': [25, 50],
                'clf__estimator__max_depth': [10, 25]
                }
cv_gb = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3, n_jobs=-1)
cv_gb.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.006293806322414533, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.006150765269632385, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=25 ...


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.9min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=25, score=0.0037195994277539344, total= 1.9min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.009726791589186096, total= 2.3min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.006007724216850236, total= 2.3min
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.0037195994277539344, total= 2.3min
[CV] clf__estimator__max_depth=25, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=25, clf__estimator__n_estimators=25, score=0.08982978114718924, total= 2.3min
[CV] clf__estimator__max_depth=25, clf__estimator__n_estimators=25 ...
[CV]  clf__estimator__max_depth=25, clf__estimator__n_estimators=25, score=0.084251180088

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 41.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [25, 50], 'clf__estimator__max_depth': [10, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [13]:
y_pred = cv_gb.predict(X_test)
clf_report_mult(y_test, y_pred)

,category,precision,recall,f1_score
0,related,0.639643,0.643402,0.641476
1,request,0.794850,0.750572,0.768433
2,offer,0.991609,0.991419,0.991514
3,aid_related,0.532319,0.528604,0.530226
4,medical_help,0.850984,0.858314,0.854603
5,medical_products,0.902605,0.908085,0.905326
6,search_and_rescue,0.946243,0.946606,0.946424
7,security,0.966438,0.966438,0.966438
8,military,0.934623,0.932685,0.933652
9,child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [14]:
with open('cv_gb.pkl', 'wb') as file:
    pickle.dump(cv_gb, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.